# 第x次实验实验报告
计算机系 计15 2021010803 郭高旭

## 实验目的
- 练习<>的计算
- 掌握使用python求<>
- 通过实例使用<>模型解决简化的实际问题
- 了解<>

## 第a题：


### 问题分析与算法设计

### 代码实现与结果
下面是代码实现，使用python实现了<>的计算

### 结果分析


### 结论



## 第b题：
### 问题分析与模型建立

### 代码实现与结果


### 结果分析

### 结论

### 意义


## 第c题：

### 问题分析与模型建立

### 代码实现与结果

### 结果分析

### 结论
